In [45]:
import numpy as np
import cv2

In [46]:
image = cv2.imread('name.jpg')
orig = image.copy()   # 원본 이미지 복사

In [47]:
# Edge Detection

r = 800.0 / image.shape[0]
dim = (int(image.shape[1] * r), 800) 
image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)  # 이미지를 resize

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # 이미지의 색공간을 BGR에서 Gray로 바꿈
gray = cv2.GaussianBlur(gray, (3, 3), 0)   #GaussianBlur를 통해 blur 효과를 줌, 외관 검출을 더 쉽게 함
edged = cv2.Canny(gray, 75, 200)  # Canny Edge Detection을 통해서 edge를 검출하게 됨

print("STEP 1: Edged Detection")  # 엣지가 검출된 이미지를 출력

cv2.namedWindow('Image', cv2.WINDOW_NORMAL)  # 윈도우 사이즈 지정
cv2.namedWindow('Edged', cv2.WINDOW_NORMAL)  # 윈도우 사이즈 지정
cv2.imshow('Image', image)
cv2.imshow('Edged', edged)

cv2.waitKey(0)
cv2.destroyAllWindows()

STEP 1: Edged Detection


In [48]:
import cv2 as cv

(cnts, _) = cv.findContours(edged.copy(), cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
    
cnts = sorted(cnts, key = cv.contourArea, reverse = True)

screenCnt = None
    
for c in cnts:
    peri = cv.arcLength(c, True)
    approx = cv.approxPolyDP(c, 0.1 * peri, True)
    
    if len(approx) == 4:
        screenCnt = approx
        break
            
print("STEP 2: Find Contours of Paper")
cv.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)
cv.imshow("Outline", image)    
        
cv.waitKey(0)
cv.destroyAllWindows()

STEP 2: Find Contours of Paper


In [49]:
import cv2 as cv

(cnts, _) = cv.findContours(edged.copy(), cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
    
cnts = sorted(cnts, key = cv.contourArea, reverse = True)[:5]
    
    
for c in cnts:
    peri = cv.arcLength(c, True)
    approx = cv.approxPolyDP(c, 0.02 * peri, True)

    if len(approx) == 4:
        screenCnt = approx
        break

    if screenCnt is None:
        detected = 0
        print ("No plate detected")
    else:
        detected = 1
    


In [50]:
def order_points(pts):
    rect = np.zeros((4, 2), dtype = "float32")
    
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    
    return rect

In [51]:
rect = order_points(screenCnt.reshape(4, 2) / r)
    
(topLeft, topRight, bottomRight, bottomLeft) = rect
    
w1 = abs(bottomRight[0] - bottomLeft[0])
w2 = abs(topRight[0] - topLeft[0])
h1 = abs(topRight[0] - bottomRight[1])
h2 = abs(topLeft[1] - bottomLeft[1])
    
maxWidth = max([w1, w2])
maxHeight = max([h1, h2])
    
dst = np.float32([[0, 0], [maxWidth-1, 0], [maxWidth-1, maxHeight-1], [0, maxHeight-1]])
    
M = cv.getPerspectiveTransform(rect, dst)
    
warped = cv.warpPerspective(orig, M, (maxWidth, maxHeight))
    
print("STEP 3: Apply perspective transform")
cv.imshow("Warped", warped)
    
cv.waitKey(0)
cv.destroyAllWindows()

<ipython-input-51-26d42125884c>:17: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  warped = cv.warpPerspective(orig, M, (maxWidth, maxHeight))


STEP 3: Apply perspective transform


error: OpenCV(4.5.1) c:\users\appveyor\appdata\local\temp\1\pip-req-build-kh7iq4w7\opencv\modules\imgproc\src\color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<3,4,-1>,struct cv::impl::A0x206ccf44::Set<1,-1,-1>,struct cv::impl::A0x206ccf44::Set<0,2,5>,2>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1


In [43]:
# import numpy as np
# import cv2 as cv

# def order_points(pts):
#     rect = np.zeros((4, 2), dtype = "float32")
    
#     s = pts.sum(axis = 1)
#     rect[0] = pts[np.argmin(s)]
#     rect[2] = pts[np.argmax(s)]
    
#     diff = np.diff(pts, axis = 1)
#     rect[1] = pts[np.argmin(diff)]
#     rect[3] = pts[np.argmax(diff)]
    
#     return rect

# def auto_scan_image():
#     image = cv.imread("opencv_test.jpg")
#     orig = image.copy()   # 원본 이미지 복사
    
#     r = 800.0 / image.shape[0]
#     dim = (int(image.shape[1] * r), 800) 
#     image = cv.resize(image, dim, interpolation = cv.INTER_AREA)  # 이미지를 resize

#     gray = cv.cvtColor(image, cv2.COLOR_BGR2GRAY)  # 이미지의 색공간을 BGR에서 Gray로 바꿈
#     gray = cv.GaussianBlur(gray, (3, 3), 0)   #GaussianBlur를 통해 blur 효과를 줌, 외관 검출을 더 쉽게 함
#     edged = cv.Canny(gray, 75, 200)  # Canny Edge Detection을 통해서 edge를 검출하게 됨

#     print("STEP 1: Edged Detection")  # 엣지가 검출된 이미지를 출력

#     cv.namedWindow('Image', cv2.WINDOW_NORMAL)  # 윈도우 사이즈 지정
#     cv.namedWindow('Edged', cv2.WINDOW_NORMAL)  # 윈도우 사이즈 지정
#     cv.imshow('Image', image)
#     cv.imshow('Edged', edged)

#     cv.waitKey(0)
#     cv.destroyAllWindows()
    
#     (_, cnts, _) = cv.findContours(edged.copy(), cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
    
#     cnts = sorted(cnts, key = cv.contourArea, reverse = True)[:5]
    
    
#     for c in cnts:
#         peri = cv.arcLength(c, True)
#         approx = cv.approxPolyDP(c, 0.02 * peri, True)
        
#         if len(approx) == 4:
#             screenCnt = approx
#             break
            
#     print("STEP 2: Find Contours of Paper")
#     cv.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)
#     cv.imshow('Outline', image)
    
#     cv.waitKey(0)
#     cv.destroyAllWIndows()
#     cv.waitKey(1)
    
#     rect = order_points(screenCnt, reshape(4, 2) / r)
    
#     (topLeft, topRight, bottomRight, bottomLeft) = rect
    
#     w1 = abs(bottomRight[0] - bottomLeft[0])
#     w2 = abs(topRight[0] - topLeft[0])
#     h1 = abs(topRight[0] - bottomRight[1])
#     h2 = abs(topLeft[1] - bottomLeft[1])
    
#     maxWidth = max([w1, w2])
#     maxHeight = max([h1, h2])
    
#     dst = np.float32([[0, 0], [maxWidhth-1, 0], [maxWidth-1], [maxHeight-1], [0. maxHeight-1]])
    
#     M = cv.getPerspectiveTransform(rect, dst)
    
#     warped = cv.warpPerspective(org, M, (maxWidth, maxHeight))
    
#     print("STEP 3: Apply perspective transform")
#     cv.imshow("Warped", warped)
    
#     cv.waitKey(0)
#     cv.destroyAllWindows()

SyntaxError: invalid syntax (<ipython-input-43-a041cb797c8d>, line 72)